# 1. Import Libary 

In [11]:
import pandas as pd
import pyodbc
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 2. Import Data Frame From SQL SERVER

In [ ]:
conn = pyodbc.connect( #CONNECTION
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

df_rfm = pd.read_sql_query('''
                                        
SELECT *
FROM [vti_dataset].[dbo].[RFM_Data]
                                       
    ; ''', conn)
df_rfm


C:\Users\Admin-s\AppData\Local\Temp\ipykernel_26424\3141880687.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rfm = pd.read_sql_query('''


,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.309998,Product C,890075,Tokyo
1,2188,2023-04-11,463.700012,Product A,176819,London
2,4608,2023-04-11,80.279999,Product A,340062,New York
3,2559,2023-04-11,221.289993,Product A,239145,London
4,9482,2023-04-11,739.559998,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.619995,Product B,275284,London
996,6669,2023-06-10,941.500000,Product C,987025,New York
997,8836,2023-06-10,545.359985,Product C,512842,London
998,1440,2023-06-10,729.940002,Product B,559753,Paris


# 3. CLEARNING DATA PROCESS

## 3.1 Remove Duplicate Rows

## 3.2 Detect & Remove Outliers Using IQR

## 3.3 Remove Irrelevant Data Using Biz Sense

## 3.4 Standardization Capitalization

## 3.5 Covert Data Type

## 3.6 Clear Format

## 3.7 Fix Error

## 3.9 Languages Transaltion

## 3.10 Handle Missing Value

# 4. Feature Engineering RFM & MIN MAX SCALERS & WEIGHT IMPORANT (R:0.2 F 0.5 M 0.3) => LABEL IT

In [12]:
df = pd.DataFrame(df_rfm)
df["PurchaseDate"] = pd.to_datetime(df["PurchaseDate"])

# -----------------------------
# Build RFM
# -----------------------------
reference_date = df["PurchaseDate"].max() + pd.Timedelta(days=1)

rfm = df.groupby("CustomerID").agg({
    "PurchaseDate": lambda x: (reference_date - x.max()).days,  # Recency
    "OrderID": "count",                                          # Frequency
    "TransactionAmount": "sum"                                   # Monetary
})

rfm.columns = ["R", "F", "M"]

# -----------------------------
# Min-Max Scaling
# -----------------------------
scaler = MinMaxScaler()
rfm_scaled = pd.DataFrame(
    scaler.fit_transform(rfm),
    columns=["R_scaled", "F_scaled", "M_scaled"],
    index=rfm.index
)

# Combine original + scaled
rfm_final = pd.concat([rfm, rfm_scaled], axis=1)

print(rfm_final)


             R  F            M  R_scaled  F_scaled  M_scaled
CustomerID                                                  
1011        34  2  1129.019974  0.550000       0.5  0.471795
1025        22  1   359.290009  0.350000       0.0  0.146647
1029         1  1   704.989990  0.000000       0.0  0.292677
1046        44  1   859.820007  0.716667       0.0  0.358080
1049        14  1   225.720001  0.216667       0.0  0.090224
...         .. ..          ...       ...       ...       ...
9941        43  1   960.530029  0.700000       0.0  0.400622
9950        39  1   679.109985  0.633333       0.0  0.281745
9954        13  1   798.010010  0.200000       0.0  0.331970
9985        58  1    36.099998  0.950000       0.0  0.010125
9991        31  1   626.809998  0.500000       0.0  0.259652

[946 rows x 6 columns]


In [13]:
rfm_final['RFM_SCORE'] = rfm_final['R_scaled'] * 0.2 + rfm_final['F_scaled'] * 0.5 + rfm_final['M_scaled'] *0.3
rfm_final

,R,F,M,R_scaled,F_scaled,M_scaled,RFM_SCORE
CustomerID,,,,,,,
1011,34,2,1129.019974,0.550000,0.5,0.471795,0.501539
1025,22,1,359.290009,0.350000,0.0,0.146647,0.113994
1029,1,1,704.989990,0.000000,0.0,0.292677,0.087803
1046,44,1,859.820007,0.716667,0.0,0.358080,0.250757
1049,14,1,225.720001,0.216667,0.0,0.090224,0.070401
...,...,...,...,...,...,...,...
9941,43,1,960.530029,0.700000,0.0,0.400622,0.260187
9950,39,1,679.109985,0.633333,0.0,0.281745,0.211190
9954,13,1,798.010010,0.200000,0.0,0.331970,0.139591


In [14]:
rfm_final['Label_RFM_SCORE'] = pd.cut(
    rfm_final['RFM_SCORE'],
    bins=[-1, 0.4, 0.7, 1],
    labels=["DELUX", "STANDARD", "VIP"]
)
rfm_final

,R,F,M,R_scaled,F_scaled,M_scaled,RFM_SCORE,Label_RFM_SCORE
CustomerID,,,,,,,,
1011,34,2,1129.019974,0.550000,0.5,0.471795,0.501539,STANDARD
1025,22,1,359.290009,0.350000,0.0,0.146647,0.113994,DELUX
1029,1,1,704.989990,0.000000,0.0,0.292677,0.087803,DELUX
1046,44,1,859.820007,0.716667,0.0,0.358080,0.250757,DELUX
1049,14,1,225.720001,0.216667,0.0,0.090224,0.070401,DELUX
...,...,...,...,...,...,...,...,...
9941,43,1,960.530029,0.700000,0.0,0.400622,0.260187,DELUX
9950,39,1,679.109985,0.633333,0.0,0.281745,0.211190,DELUX
9954,13,1,798.010010,0.200000,0.0,0.331970,0.139591,DELUX


# 5. Save Clear Data Into Database To Visiualization ti

In [15]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "use_case_3_RFM_Normalization" #

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(rfm_final[c].dtype)}" for c in rfm_final.columns]) #
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(rfm_final.columns))})", rfm_final.values.tolist()) #
conn.commit()

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# -----------------------------
# Sample raw data
# -----------------------------
df = pd.read_csv(r"C:\Users\Admin-s\Downloads\Use_Case_Normalization\1.data.csv")
# Keep only R F M
rfm = df[["R", "F", "M"]].copy()

# ------------------------------------
# Min-Max Scaling
# ------------------------------------
scaler = MinMaxScaler()
rfm_scaled = pd.DataFrame(
    scaler.fit_transform(rfm),
    columns=["R_scaled", "F_scaled", "M_scaled"]
)

# ------------------------------------
# Apply weights
# R: 0.2, F: 0.4, M: 0.4
# ------------------------------------
rfm_scaled["RFM_score"] = (
    rfm_scaled["R_scaled"] * 0.2 +
    rfm_scaled["F_scaled"] * 0.4 +
    rfm_scaled["M_scaled"] * 0.4
)

# ------------------------------------
# Combine with original names
# ------------------------------------
final_df = pd.concat([df["Name"], rfm, rfm_scaled], axis=1)

print(final_df)
